In [10]:
import os
from collections import deque

from MolJuncTree import MolJuncTree  
from ClusterVocab import ClusterVocab 

import torch
import torch.nn as nn

In [11]:
x = torch.tensor([x for x in range(5)])

In [14]:
y = [x for i in range(5)]

In [16]:
y = torch.stack(y, dim=0)

In [18]:
y.shape

torch.Size([5, 5])

In [19]:
z = torch.sum(y, dim=0)

In [20]:
z

tensor([ 0,  5, 10, 15, 20])

In [2]:
VOCAB_PATH = os.path.join(os.getcwd(), 'data', 'vocab_50.txt')
vocab = [x.strip("\r\n ") for x in open(VOCAB_PATH)]
vocab = ClusterVocab(vocab)

hidden_size = 3

embedding = nn.Embedding(vocab.size(), hidden_size)

In [9]:
def get_bottom_up_traversal_order(root):
    # FIFO queue for BFS traversal
    fifo_queue = deque([root])

    # set to keep track of visited nodes
    visited = set([root.idx])
    
    # root node is at zeroth depth
    root.depth = 0

    # list to store appropriate traversal order
    bottom_up = []

    while len(fifo_queue) > 0:
        # pop node from front of the queue
        x = fifo_queue.popleft()

        # traverse the neighbors
        for y in x.neighbors:
            if y.idx not in visited:
                fifo_queue.append(y)

                visited.add(y.idx)

                y.depth = x.depth + 1

                if y.depth > len(bottom_up):
                    # have a separate sublist for every depth
                    bottom_up.append([])
                
                bottom_up[y.depth - 1].append((y, x))

    # first we implement bottom-up traversal and then top-down traversal
    traversal_order = bottom_up[::-1]

    return traversal_order


In [10]:
traversal_order = get_bottom_up_traversal_order(root_batch[0])

In [11]:
next_val = [torch.zeros(hidden_size)] * len(junc_tree_batch[0].nodes)

In [13]:
next_val = torch.stack(next_val, dim=0)

In [14]:
next_val.shape

torch.Size([19, 3])

In [23]:
A = torch.randn(hidden_size, hidden_size)
B = torch.randn(hidden_size, hidden_size)
V = torch.randn(hidden_size, hidden_size)
U = torch.randn(hidden_size, hidden_size)

In [26]:
MAX_NEIGHBORS = 8

padding = torch.zeros(hidden_size)

for iter in range(len(traversal_order)):
    edge_tuple_list = traversal_order[iter]
    
    # all hidden vectors, of child nodes, for this iteration 
    child_hidden_vecs_iter = []
    
    edges_gates_iter = []
    
    parent_node_idx = []
    
    for node_x, node_y in edge_tuple_list:
        
        children = [node_z.idx for node_z in node_x.neighbors if node_z.idx != node_y.idx]
        
        print(str(node_x.global_idx) + " -> " + str(children))
        
        parent_node_idx.append(node_x.global_idx)

        child_hidden_vecs = []

        if len(children) > 0:
            child_hidden_vecs = list(torch.index_select(initial_values, 0, torch.LongTensor(children)))
 
        pad_len = MAX_NEIGHBORS - len(child_hidden_vecs)
        child_hidden_vecs.extend([padding] * pad_len)
        
        child_hidden_vecs_iter.extend(child_hidden_vecs)
        edges_gates_iter.extend(child_hidden_vecs)
    
    x = torch.stack(child_hidden_vecs_iter, dim=0)
    child_hidden_vecs_iter = torch.mm(x, V)
    child_hidden_vecs_iter = child_hidden_vecs_iter.view(-1, MAX_NEIGHBORS, hidden_size)
    
    y = torch.stack(edges_gates_iter, dim=0)
    edges_gates_iter = torch.mm(y, B)
    edges_gates_iter = edges_gates_iter.view(-1, MAX_NEIGHBORS, hidden_size)
    
    parent_node_hidden_vecs = torch.index_select(initial_values, 0, torch.tensor(parent_node_idx))
    
    if len(edge_tuple_list) > 2:
        break
    
    
    # 
    # child_hidden_vecs_iter_sum = torch.sum(child_hidden_vecs_iter, dim=1)

11 -> []
12 -> []
18 -> [11, 12]
9 -> []
10 -> [18]
13 -> []


In [27]:
x

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.7539, -0.9501, -1.1129],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<StackBackward>)

In [28]:
child_hidden_vecs_iter

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-1.5638, -3.8015, -1.4700],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<ViewBackward>)

In [29]:
y

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.7539, -0.9501, -1.1129],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<StackBackward>)

In [30]:
y

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.7539, -0.9501, -1.1129],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<StackBackward>)

In [31]:
edges_gates_iter

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.6203,  1.6438, -0.8755],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<ViewBackward>)

In [32]:
parent_node_hidden_vecs

tensor([[-0.2784,  0.6788,  0.2787],
        [-0.1116, -0.5129, -0.5221],
        [-0.2784,  0.6788,  0.2787]], grad_fn=<IndexSelectBackward>)

In [35]:
q = torch.mm(parent_node_hidden_vecs, A)

In [36]:
q

tensor([[ 0.4184, -0.0385,  0.1612],
        [-0.4550,  0.2893, -0.4657],
        [ 0.4184, -0.0385,  0.1612]], grad_fn=<MmBackward>)

In [37]:
edges_gates_iter.shape

torch.Size([3, 8, 3])

In [39]:
e = q.unsqueeze(1)

In [40]:
e.shape

torch.Size([3, 1, 3])

In [41]:
g = edges_gates_iter + e

In [47]:
h = nn.Sigmoid()(g)

In [48]:
h

tensor([[[0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402]],

        [[0.5412, 0.8736, 0.2073],
         [0.3882, 0.5718, 0.3856],
         [0.3882, 0.5718, 0.3856],
         [0.3882, 0.5718, 0.3856],
         [0.3882, 0.5718, 0.3856],
         [0.3882, 0.5718, 0.3856],
         [0.3882, 0.5718, 0.3856],
         [0.3882, 0.5718, 0.3856]],

        [[0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402],
         [0.6031, 0.4904, 0.5402]]], grad_fn=<SigmoidBackward>)

In [49]:
g

tensor([[[ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612]],

        [[ 0.1652,  1.9331, -1.3412],
         [-0.4550,  0.2893, -0.4657],
         [-0.4550,  0.2893, -0.4657],
         [-0.4550,  0.2893, -0.4657],
         [-0.4550,  0.2893, -0.4657],
         [-0.4550,  0.2893, -0.4657],
         [-0.4550,  0.2893, -0.4657],
         [-0.4550,  0.2893, -0.4657]],

        [[ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612],
         [ 0.4184, -0.0385,  0.1612]]], grad_fn=<AddBackward0>)